## Improving RAG using LangGraph and LangChain

In [ ]:
! pip install langchain langchain-community langgraph Chroma sentence_transformers chromadb

In [17]:
from typing import Dict , TypedDict , Optional
from langgraph.graph import StateGraph  , END
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings

from google.colab import userdata

hg_api =  userdata.get('hugginface_key')

import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hg_api
from langchain.llms import HuggingFaceHub
llm = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha", model_kwargs={"temperature":0.5, "max_length":512})



## Defining StateGraph

In [3]:
class GraphState(TypedDict) :
  question: Optional[str] = None
  classification :Optional[str] = None
  response : Optional[str] = None
  length : Optional[int] = None
  greeting : Optional[str] = None

workflow =   StateGraph(GraphState)



## What’s a StateGraph?

The heart of any LangGraph flow, StateGraph stores the state of various variables we would be storing while executing the workflow. In this case, we have 5 variables whose values we would be updating while executing the graph and would be shared with all edges and nodes.

In [4]:
def retriever_qa_creation():
  embeddings =  HuggingFaceEmbeddings()
  db = Chroma(persist_directory="./data/" , embedding_function= embeddings , collection_name="data")
  retriever = db.as_retriever()
  return RetrievalQA.from_chain_type(llm=llm , chain_type="stuff" , retriever=retriever)

rag_chain =  retriever_qa_creation()

<ipython-input-4-f167433e3385>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings =  HuggingFaceEmbeddings()
<ipython-input-4-f167433e3385>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings =  HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggi

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-4-f167433e3385>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="./data/" , embedding_function= embeddings , collection_name="data")


In [5]:
def classify(question) :
  return llm("Classify intent of given input as greering or not_greeting, Ouptut just class: {}".format(question)).strip()

In [6]:
def classify_input_node(state) :
  question = state.get('question' , '').strip()
  classification = classify(question)
  return {"classification" : classification}


In [7]:
def handle_greeting_node(state) :
  return {"greeting" : "Hello ! How can i help you today ?"}

In [8]:
def handle_RAG(state) :
  question = state.get('question' , '').strip()
  prompt = question
  if state.get("length") < 30 :
    search_result = rag_chain.run(prompt)
  else :
    search_result = rag_chain.run(prompt +'. Return total count only.')
  return {"response" : search_result , "length" : len(search_result)}

def bye(state) :
    return{"greeting":"The graph has finished"}


In [9]:
workflow.add_node("classify_input" , classify_input_node)
workflow.add_node("handle_greeting" , handle_greeting_node)

In [10]:
workflow.add_node("handle_RAG" ,handle_RAG)

In [11]:
workflow.add_node("bye" , bye)

In [12]:
workflow.set_entry_point("classify_input")
workflow.add_edge('handle_greeting', END)
workflow.add_edge('bye', END)

In [13]:
def decide_next_node(state):
    return "handle_greeting" if state.get('classification') == "greeting" else "handle_RAG"

def check_RAG_length(state):
    return "handle_RAG" if state.get("length")>30 else "bye"

workflow.add_conditional_edges(
    "classify_input",
    decide_next_node,
    {
        "handle_greeting": "handle_greeting",
        "handle_RAG": "handle_RAG"
    }
)

workflow.add_conditional_edges(
    "handle_RAG",
    check_RAG_length,
    {
        "bye": "bye",
        "handle_RAG": "handle_RAG"
    }
)

# A conditional edge helps to choose between 2 nodes depending upon a condition (say if-else). In the 2 conditional edges created:

### 1st conditional edge

- Onec “classifiy_input” is encountered, choose either “handle_greeting” or “handle_RAG” depending upon the output of decide_next_node function

### 2nd conditional edge

- If “handle_RAG” is encountered, choose either “handle_RAG” or “bye” depending upon check_RAG_length.

In [ ]:
app = workflow.compile()
app.invoke({'question':'Mehul developed which projects?','length':0})

The graph flow looks something like this for the above prompt

classify_input : The sentiment would be not_greeting

Due to 1st conditional_edge, moves to handle_RAG

As length=0, use 1st prompt and retrieve answer (total length would be>30)

Due to 2nd condtional_edge, moves again to handle_RAG

As length>30, use 2nd prompt

Due to 2nd conditional_edge, moves to bye

END

In [ ]:
rag_chain.run("Mehul developed which projects?")


In [ ]:
app.invoke({'question':'Hello bot','length':0})


The flow here would be simpler

classify_input : The sentiment would be “greeting”

Due to 1st conditional_edge, moves to handle_greeting

END|